In [21]:
import torch
import torch.nn as nn
import numpy as np
import transformers
from transformers import DistilBertTokenizer, DistilBertModel

In [22]:
device ='cpu'
MAX_LEN = 512

In [23]:
class DistilBERTClass(nn.Module):
    
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(768, 6)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: dfb03fb1-8787-45ff-909a-0f1e175caf7a)')' thrown while requesting HEAD https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt


In [24]:
toxic_model = DistilBERTClass().to(device)
toxic_model.load_state_dict(torch.load('model_0.018.pth', map_location=torch.device('cpu')))
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 9b9abf58-c767-44c6-af5c-0d3c2d1163ce)')' thrown while requesting HEAD https://huggingface.co/distilbert-base-uncased/resolve/main/config.json


<All keys matched successfully>

In [55]:
text = 'I WILL KILL YOU'

with torch.no_grad():
    cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    inputs = tokenizer.encode_plus(text, None, add_special_tokens=True, max_length=MAX_LEN,
                                    pad_to_max_length=True, return_token_type_ids=True,)

    output = {
        'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
        'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
        'token_type_ids': torch.tensor(inputs["token_type_ids"], dtype=torch.long)
    }

    result = nn.Sigmoid()(toxic_model(output['ids'], output['mask'], output['token_type_ids']))

print(cols)
print(result>0.3)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
tensor([[ True, False, False,  True, False, False]])
